## M3E-small模型

In [8]:
# 导入sentence_transformers
import json
import pdfplumber
from sentence_transformers import SentenceTransformer

In [14]:
# 加载数据集，问题与知识源
questions = json.load(open("../data/Coggle比赛数据/汽车知识问答/questions.json"))
pdf = pdfplumber.open("../data/Coggle比赛数据/汽车知识问答/初赛训练数据集.pdf")
pdf.pages
pdf.pages[0].extract_text()
# 此工具会将pdf分页，并提取每页的内容

'欢迎\n感谢您选择了具有优良安全性、舒适性、动力性和经济性的Lynk&Co领克汽车。\n首次使用前请仔细、完整地阅读本手册内容，将有助于您更好地了解和使用车辆。\n本手册中的所有资料均为出版时的最新资料，但本公司将对产品进行不断的改进和优化，您所购的车辆可能与本手册中的描述有所不同，请以实际\n接收的车辆为准。\n如您有任何问题，或需要预约服务，请拨打电话4006-010101联系我们。您也可以开车前往Lynk&Co领克中心。\n在抵达之前，请您注意驾车安全。\n©领克汽车销售有限公司'

In [10]:
# 将pdf切分后的东西，将页码和内容组成json,并放在list中
pdf_content = []
for page_idx in range(len(pdf.pages)):
    pdf_content.append({
        'page':'page_'+str(page_idx+1),
        'content':pdf.pages[page_idx].extract_text()
    })
pdf_content[:1]


[{'page': 'page_1',
  'content': '欢迎\n感谢您选择了具有优良安全性、舒适性、动力性和经济性的Lynk&Co领克汽车。\n首次使用前请仔细、完整地阅读本手册内容，将有助于您更好地了解和使用车辆。\n本手册中的所有资料均为出版时的最新资料，但本公司将对产品进行不断的改进和优化，您所购的车辆可能与本手册中的描述有所不同，请以实际\n接收的车辆为准。\n如您有任何问题，或需要预约服务，请拨打电话4006-010101联系我们。您也可以开车前往Lynk&Co领克中心。\n在抵达之前，请您注意驾车安全。\n©领克汽车销售有限公司'}]

In [17]:
# 解析question,与pdf中对应的内容（不包括page页码）到list中
question_sentences = [x['question'] for x in questions]
pdf_content_sentences = [x['content'] for x in pdf_content]

pdf_content_sentences[-5]

'技术资料\n动力性能 座椅参数\n最大设计车 最大爬坡度 项目 前排座椅调节 后排座椅参数\n车型 驱动方式\n速（km/h） （a）（%）\n设定的座椅前后位 向前滑动210mm， 不可调节\nMR6453DCH- 置（a） 向后滑动50mm\n两驱 200 40\nEV02\n设定的靠背角状\n（a）：车辆以坡底起步。 态（a） 25° 中间22°，两侧26°\n从设计角度向前调\n油耗 座椅靠背的正常使\n整26°，向后调整 不可调节\n用状态\n与车辆认证驾驶循环相比（下表所列油耗以车辆认证驾驶循环为基 45°\n础），实际油耗可能略有变化。下列参数会影响油耗和排放：车辆载 （a）：测量座垫深时。\n荷、轮胎气压、驾驶风格、车速和加速度。\n车轮定位参数\nWLTC亏电油耗（L/100\n车型\nkm） 车轮定位 数据\nMR6453DCHEV02 5.07 前轮外倾角 -0.74°±0.65°\n后轮外倾角 -0.73°±0.73°\n燃油规格\n主销内倾角 14.20°±0.50°\n燃油 油箱容积 等级\n主销后倾角 4.80°±0.50°\n汽油 42L 92号及以上\n前车轮前束 0.11°±0.05°\n注意！ 后车轮前束 0.10°±0.05°\n■ 因使用劣质燃油或燃油中掺入非Lynk&Co领克认证的添加剂而引\n起的损坏或故障不在Lynk&Co领克的保修范围内。\n350'

In [5]:
# 加载embedding 模型

model = SentenceTransformer("moka-ai/m3e-small")

In [6]:
# 将问题与pdf中的所有回答进行向量编码
question_embeddings = model.encode(question_sentences,normalize_embeddings=True)
pdf_embeddings = model.encode(pdf_content_sentences,normalize_embeddings=True)

d:\ProgramData\miniconda3\envs\big_model\lib\site-packages\transformers\models\bert\modeling_bert.py:439: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


In [7]:
for query_idx,feat in enumerate(question_embeddings):
    score = feat@pdf_embeddings.T  # 将question中embedding后的每一个问题在pdf_embeddings中做相似度匹配，并打分
    max_score_page_idx = score.argsort()[-1]+1 #这是因为内容页的最后都会给出页码
    questions[query_idx]['reference']='page_'+str(max_score_page_idx)

questions[0]

{'question': '“前排座椅通风”的相关内容在第几页？', 'answer': '', 'reference': 'page_116'}